In [1]:
from ete3 import Tree
import pandas as pd
import glob
from Bio import SeqIO
import random
from random import choices

In [2]:
### Function for getting closest relatives of one leaf 
### (It means getting all descendants of a parental node of a leaf)
### Not used in the end
def getRelatedLeafs(node):
    nodeup = node.up
    n = nodeup.get_descendants()
    N = []
    for nod in n:
        if nod.is_leaf():
            N.append(nod.name)
    return N, nodeup

In [32]:
### Reading population info and IR coordinates on the genome (.bed)
samples_fn = "./data/Species_distinction_PCA.txt"
df = pd.read_csv(samples_fn,sep='\t',header=0)
D = {a: [b,c] for a,b,c in zip(list(df["SpName"]),list(df["GroupSamtools"]),list(df["SamName"]))}
bed = pd.read_csv("./data/strain_merged_all_location.bed", sep = "\t", header = None, 
                  names = ["chrom","start","stop","strain","lineage","location","region"],
                 dtype={"start": "Int64", "stop": "Int64"})
ulmi = df.loc[df["GroupSamtools"] == "U","SamName"].values
ulm = list(ulmi)

In [24]:
### Reading trees generated for each IR
trees = glob.glob("./raxml_trees/RAxML_bipartitions_75coll.si_*")

In [25]:
### For each ONU strain with IR select all OU strains
F = []
for tree in trees:
    strain = tree.split("_")[6]
    region = "_".join(tree.split("_")[4:6])
    #print(strain, region)
    #t = Tree(tree,format=2)
    #t.get_leaf_names()
    #strainName = D[strain][1]
    #strainNode = t&strainName
    #relatives, uppernode = getRelatedLeafs(strainNode)
    #leaf_nodes = [ele for ele in relatives if ele != strainName]
    chrom = bed[(bed["start"] == int(region.split("_")[0])) & (bed["stop"] == int(region.split("_")[1])) & (bed["strain"] == strain)]
    f = strain, list(chrom['chrom'])[0], region.split("_")[0], region.split("_")[1], ','.join(ulm)
    F.append(f)
F[:2]

[('TR88',
  'OphioH327chr_6',
  '1600000',
  '2800000',
  '785401_U,H1018_U,H1021_U,H1022_U,H1096_U,H2062_U,H337_U,H359_U,H379_U,H649_U,H809_U,H864_U,H868_U,H871_U,H872_U,H873_U,H909_U,H924_U,I131_U,I185_U,PG40_U'),
 ('R108',
  'OphioH327chr_6',
  '400000',
  '600000',
  '785401_U,H1018_U,H1021_U,H1022_U,H1096_U,H2062_U,H337_U,H359_U,H379_U,H649_U,H809_U,H864_U,H868_U,H871_U,H872_U,H873_U,H909_U,H924_U,I131_U,I185_U,PG40_U')]

In [9]:
### Function for filtering sequences with too much missing data
def filterBadSeq(BigList):
    NewList = []
    for sek in BigList:
        length = len(sek)
        nmissi = sek.count('N')
        propmi = nmissi/float(length)
        if propmi > 0.75:
            continue
        else:
            NewList.append(sek)
    return NewList

In [10]:
### Reading fasta alignments of IRs; for each introgressed ONU removing poor quality sequences 
### and selecting only the this ONU strain, one OU sequences, and O. himal-ulmi sequences (outgroup)
T = {}
for ele in F:
    ulmi = ele[-1].split(",")
    fnames = glob.glob("./fasta/si_"+ele[2]+"_"+ele[3]+"_"+ele[0]+"_*.fasta")
    N,O,U = [],[],[]
    for record in SeqIO.parse(fnames[0],"fasta"):
        if record.id == D[ele[0]][1]:
            N.append(str(record.seq).upper())
        elif record.id in ["HP30_H","HP31_H","HP32_H"]:
            O.append(str(record.seq).upper())
        elif record.id in ulmi:
            U.append(str(record.seq).upper())
    
    dN = filterBadSeq(N)
    booln = 1
    if dN == []:
        print(ele[0]+"_"+ele[1]+"_"+ele[2]+"_"+ele[3]+" novo-ulmi too bad")
        booln = 0
    dO = filterBadSeq(O)
    if dO == []:
        print(ele[0]+"_"+ele[1]+"_"+ele[2]+"_"+ele[3]+" no outgroup")
        booln = 0
    dU = filterBadSeq(U)
    if dU == []:
        print(ele[0]+"_"+ele[1]+"_"+ele[2]+"_"+ele[3]+" no ulmi strain")
        booln = 0
    
    if booln == 1:
        outgroup_tuple = list(zip(*dO))
        focal_nt = list(dN[0])
        ulmi_tuple = list(zip(*dU))
        T[ele[0]+"_"+ele[1]+"_"+ele[2]+"_"+ele[3]] = outgroup_tuple, focal_nt, ulmi_tuple
        
#list(T.keys())[:4]
#T['TR88_OphioH327chr_6_1600000_2800000']

In [20]:
### For each IR, after removing 20 kb on both sides of the IR, select random bases for OU and outgroup,
### and encode each base as a 3-number code
TN = {}
AS = {}
edges_correction=True
for ele in list(T.keys()):
    print(ele)
    
    if edges_correction == True:
        outgroup = T[ele][0][20000:-20000]
        focal = T[ele][1][20000:-20000]
        ulmi = T[ele][2][20000:-20000]
        N010,N001, N, N011 = [],[],[],[]
        t = []
    else:
        outgroup = T[ele][0]
        focal = T[ele][1]
        ulmi = T[ele][2]
        N010,N001, N, N011 = [],[],[],[]
        t = []
    pats = []
    for nt in range(len(focal)):
        out_flt = [i for i in list(outgroup[nt]) if i != 'N']
        focal_flt = [i for i in [focal[nt]] if i != 'N']
        ulmi_flt = [i for i in list(ulmi[nt]) if i != 'N']
        if (out_flt!=[]) & (focal_flt!=[]) & (ulmi_flt!=[]):
            N.append(1)
            out = random.sample(out_flt,1)[0]
            foc = focal_flt[0]
            ulm = random.sample(ulmi_flt,1)[0]
            if (out == ulm) & (out!=foc):
                N010.append(1)
                pats.append('010')
            elif (out != ulm) & (out==foc):
                N001.append(1)
                pats.append('001')
            elif (out != ulm) & (out != foc):
                N011.append(1)
                pats.append('011')
            else:
                pats.append('000')
        t.append([out,foc,ulm])

    TN[ele] = sum(N),sum(N010),sum(N001), sum(N011)
    AS[ele] = pats

TR88_OphioH327chr_6_1600000_2800000
R108_OphioH327chr_6_400000_600000
R108_OphioH327chr_3_0_200000
V19_OphioH327chr_7_0_200000
Yu16_OphioH327chr_1_0_200000
H371_OphioH327chr_1_0_400000
H986_OphioH327chr_1_0_400000
P122_OphioH327chr_1_0_400000
V19_OphioH327chr_1_0_400000
TR88_OphioH327chr_1_1600000_2200000
H413a_OphioH327chr_8_2000000_2600000
R22_OphioH327chr_1_200000_400000
P122_OphioH327chr_4_2200000_2400000
V19_OphioH327chr_7_2200000_2400000
H323_OphioH327chr_5_2400000_2600000
TR88_OphioH327chr_3_3600000_3800000
V19_OphioH327chr_3_3600000_3800000
H276_OphioH327chr_6_400000_600000
P114_OphioH327chr_5_400000_600000
P122_OphioH327chr_6_400000_600000
R22_OphioH327chr_6_400000_600000
H2098_OphioH327chr_2_4800000_5800000
H2091_OphioH327chr_2_5200000_5600000
Yu16_OphioH327chr_2_5200000_5600000
DDS298_OphioH327chr_2_5400000_5600000
DDS93_OphioH327chr_2_5400000_5600000
US137_OphioH327chr_2_5400000_5600000
H413a_OphioH327chr_2_5400000_6800000
H986_OphioH327chr_2_5800000_6800000
R108_OphioH327c

In [34]:
### Saving average branch length estimates
bD = {}
for ele in TN.keys():
    bD[ele] = "_".join(ele.split("_")[1:3]),int(ele.split("_")[3]),int(ele.split("_")[4]),ele.split("_")[0],TN[ele][0],TN[ele][1],TN[ele][2],TN[ele][3]

x1,x2,x3,x4,x5,x6,x7,x8 = list(zip(*list(bD.values())))
dbd = pd.DataFrame({"chrom":x1,"start":x2,"stop":x3,"strain":x4,"allSites":x5,"focSites":x6,"ulmSites":x7,"fixedHIM":x8})
dx = pd.merge(dbd,bed,on = ["chrom","start","stop","strain"], how = "left")
dx["Length"]=dx["stop"]-dx["start"]
dx["ttout"]=((dx["focSites"] + dx["ulmSites"])/2)/dx["fixedHIM"]
#dx.head()
dx.to_csv("./data/selectStrainsForbranchEstimationAllOU.out",sep = "\t", header=True, index=False)

In [27]:
### Run 100 empirical bootstraps of base patters for each IR
boot = 100
BT = {}
for ele in list(AS.keys()):
    print(ele)
    A = len(AS[ele])
    L = A
    #L = minlen
    Boot = []
    for bootstrap in range(boot):
        bootsites = random.choices(AS[ele],k=int(L))
        n010 = bootsites.count("010")
        n001 = bootsites.count("001")
        n011 = bootsites.count("011")
        n000 = bootsites.count("000")
        tot = n010 + n001 + n011 + n000
        ttout = ((n010+n001)/2)/n011
        Boot.append([bootstrap,n010,n001,n011,n000,tot,ttout])
    BT[ele] = Boot

TR88_OphioH327chr_6_1600000_2800000
R108_OphioH327chr_6_400000_600000
R108_OphioH327chr_3_0_200000
V19_OphioH327chr_7_0_200000
Yu16_OphioH327chr_1_0_200000
H371_OphioH327chr_1_0_400000
H986_OphioH327chr_1_0_400000
P122_OphioH327chr_1_0_400000
V19_OphioH327chr_1_0_400000
TR88_OphioH327chr_1_1600000_2200000
H413a_OphioH327chr_8_2000000_2600000
R22_OphioH327chr_1_200000_400000
P122_OphioH327chr_4_2200000_2400000
V19_OphioH327chr_7_2200000_2400000
H323_OphioH327chr_5_2400000_2600000
TR88_OphioH327chr_3_3600000_3800000
V19_OphioH327chr_3_3600000_3800000
H276_OphioH327chr_6_400000_600000
P114_OphioH327chr_5_400000_600000
P122_OphioH327chr_6_400000_600000
R22_OphioH327chr_6_400000_600000
H2098_OphioH327chr_2_4800000_5800000
H2091_OphioH327chr_2_5200000_5600000
Yu16_OphioH327chr_2_5200000_5600000
DDS298_OphioH327chr_2_5400000_5600000
DDS93_OphioH327chr_2_5400000_5600000
US137_OphioH327chr_2_5400000_5600000
H413a_OphioH327chr_2_5400000_6800000
H986_OphioH327chr_2_5800000_6800000
R108_OphioH327c

In [28]:
### Create data frame with bootstrap results
R = []
for ele in BT.keys():
    strain = ele.split("_")[0]
    chrom = "_".join(ele.split("_")[1:3])
    start = ele.split("_")[3]
    stop = ele.split("_")[4]
    length = int(stop) - int(start)
    for boot in BT[ele]:
        r = [strain, chrom, int(start), int(stop), str(length)] + boot
        R.append(r)
Rzip = list(zip(*R))
dr = pd.DataFrame({"strain":Rzip[0],"chrom":Rzip[1],"start":Rzip[2],"stop":Rzip[3],"length":Rzip[4],"boot":Rzip[5],"n010":Rzip[6],"n001":Rzip[7],"n011":Rzip[8],"n000":Rzip[9],"tot":Rzip[10],"ttout":Rzip[11]})
dr.head()

,strain,chrom,start,stop,length,boot,n010,n001,n011,n000,tot,ttout
0,TR88,OphioH327chr_6,1600000,2800000,1200000,0,1491,1694,47552,904819,955556,0.033490
1,TR88,OphioH327chr_6,1600000,2800000,1200000,1,1558,1717,47400,904881,955556,0.034546
2,TR88,OphioH327chr_6,1600000,2800000,1200000,2,1483,1743,47569,904761,955556,0.033909
3,TR88,OphioH327chr_6,1600000,2800000,1200000,3,1509,1694,47390,904963,955556,0.033794
4,TR88,OphioH327chr_6,1600000,2800000,1200000,4,1506,1658,47687,904705,955556,0.033175


In [30]:
### Append info about location number
dm = pd.merge(dr, bed, on = ["chrom","start","stop"])
dm.head()

,strain_x,chrom,start,stop,length,boot,n010,n001,n011,n000,tot,ttout,strain_y,lineage,location,region
0,TR88,OphioH327chr_6,1600000,2800000,1200000,0,1491,1694,47552,904819,955556,0.033490,TR88,NOV,12,10
1,TR88,OphioH327chr_6,1600000,2800000,1200000,1,1558,1717,47400,904881,955556,0.034546,TR88,NOV,12,10
2,TR88,OphioH327chr_6,1600000,2800000,1200000,2,1483,1743,47569,904761,955556,0.033909,TR88,NOV,12,10
3,TR88,OphioH327chr_6,1600000,2800000,1200000,3,1509,1694,47390,904963,955556,0.033794,TR88,NOV,12,10
4,TR88,OphioH327chr_6,1600000,2800000,1200000,4,1506,1658,47687,904705,955556,0.033175,TR88,NOV,12,10


In [28]:
### Save results to table
dm.to_csv("./data/selectStrainsForbranchEstimationBootstrapAllOU.out",sep = "\t", header=True, index=False)